In [44]:
import pandas as pd
import os
import glob
from tqdm import tqdm
import re
import numpy as np
import tensorflow as tf

In [45]:
# create repo

folders = os.listdir('/home/cgiuser/MK/learning/LSTM/data/maildir')
final_mails = []
for f_name in tqdm(folders):
    text_dir = f'/home/cgiuser/MK/learning/LSTM/data/maildir/{f_name}/inbox/*.*'
    file_names = glob.glob(text_dir)
    final_mails.extend(file_names)


100%|██████████| 150/150 [00:00<00:00, 982.92it/s]


In [46]:
final = []
for mail in tqdm(final_mails):
    try:
        with open(mail,'r',encoding='utf-8') as f:
            check = f.read()
        msgs = check.split(' -----Original Message-----\n')

        for i in range(len(msgs)-1):
            if i == 0:
                final.append([msgs[i].split('.pst')[-1],msgs[i+1].split('Subject:')[-1]])
            else:
                final.append([msgs[i].split('Subject:')[-1],msgs[i+1].split('Subject:')[-1]])
    except:
        pass

    
email_data = pd.DataFrame(final,columns=['Sentence_1','Sentence_2'])

email_data.head(5)

100%|██████████| 41509/41509 [00:01<00:00, 27829.07it/s]


,Sentence_1,Sentence_2
0,\n\nI'll call him but the answer is ..the same...,"\t\n\nHey, got a call from Jeff Smiron. Appare..."
1,\n\nChip - please put the following transactio...,Centana Storage Arrangement\n\n\n\nChris/Joe\...
2,\n\nThe only one I have is for Saturday. It is...,\tRE: Feb Gas Sales\n\ncool. have you gotten ...
3,\tRE: Feb Gas Sales\n\ncool. have you gotten ...,\tRE: Feb Gas Sales\n\nHe has already called m...
4,\tRE: Feb Gas Sales\n\nHe has already called m...,\tFeb Gas Sales\n\nKen Fowler is the man at TX...


In [47]:

# https://stackoverflow.com/questions/43018030/replace-apostrophe-short-words-in-python
# https://stackoverflow.com/questions/49073673/include-punctuation-in-keras-tokenizer

def nlp_preprocessing(total_text):
    if type(total_text) is not int:
        
        result1 = total_text.find('.')
        result2 = total_text.find('?')
        if result1 != -1 and result2 != -1:
            limit = min(result1,result2)
        elif result1 != -1 and result2 == -1:
            limit = result1
        else:
            limit = result2

        total_text =  total_text[0:limit+1]
        # replace multiple spaces with single space
        total_text = re.sub('\s+', ' ', total_text)
#         total_text = re.sub('\s+',' ', total_text)
        
        #This is to be done because I want to Include Period and Question mark in keras tokenizer.
        #I do not want the Tokenizer API to remove them
        total_text = total_text.replace(".", " .")
        total_text = total_text.replace("?", " ?")
        
        # specific
        total_text = re.sub(r"won\'t", "will not", total_text)
        total_text = re.sub(r"can\'t", "can not", total_text)

        # general
        total_text = re.sub(r"n\'t", " not", total_text)
        total_text = re.sub(r"\'re", " are", total_text)
        total_text = re.sub(r"\'s", " is", total_text)
        total_text = re.sub(r"\'d", " would", total_text)
        total_text = re.sub(r"\'ll", " will", total_text)
        total_text = re.sub(r"\'t", " not", total_text)
        total_text = re.sub(r"\'ve", " have", total_text)
        total_text = re.sub(r"\'m", " am", total_text)
        # converting all the chars into lower-case.
        total_text = total_text.lower().strip()
        
        
        return total_text
    
email_data['Sentence_1'] = email_data['Sentence_1'].apply(nlp_preprocessing)
email_data['Sentence_2'] = email_data['Sentence_2'].apply(nlp_preprocessing)

email_data.head(2)

,Sentence_1,Sentence_2
0,i will call him but the answer is .,"hey, got a call from jeff smiron ."
1,chip - please put the following transaction in...,centana storage arrangement chris/joe attached...


In [48]:
email_data.Sentence_2.iloc[0]

'hey, got a call from jeff smiron .'

In [49]:
email_data.shape

(13674, 2)

In [7]:
# Removing the rows having Question or Answer length (character length) > 1000 characters
list_index = []
for ind in email_data.index:
    if len(email_data['Sentence_1'][ind]) > 1000 or len(email_data['Sentence_2'][ind]) > 1000:
        list_index.append(ind)
    
# Dropping the above calculated Indexes
email_data.drop(list_index, inplace=True)
email_data.shape

(13662, 2)

In [8]:
answers = email_data.Sentence_1.to_list()
questions = email_data.Sentence_2.to_list()

In [9]:
#PRARAMBH means the begenning of a Sentence 
#SAMAAPT means the end of a sentence
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
answers_with_tags = list()
for i in range(len(answers)) :

    answers_with_tags.append('PRARAMBH '+ answers[i] +' SAMAAPT')


# I want the Tokenizer to remove all speacial characters except "." and "?"
tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;<=>@[\\]^_`{|}~\t\n') #All without "." and "?"
tokenizer.fit_on_texts(questions + answers_with_tags )

# word_index is a dixtionary containing word and their index
# adding 1 to VOCAB_SIZE because word_index starts from 0
VOCAB_SIZE = len(tokenizer.word_index)+1

In [10]:
VOCAB_SIZE

30094

In [11]:
# Creating data from Encodder Input - encoder_input_data

# Convert Question to Sequence
tokenized_questions = tokenizer.texts_to_sequences(questions)

# Get the max length of Question after tokenization
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )

# Do Post padding
padded_questions = tensorflow.keras.utils.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
# Print shapes
print( encoder_input_data.shape , maxlen_questions )

(13662, 55) 55


In [12]:
tokenized_answers = tokenizer.texts_to_sequences( answers_with_tags )

for i in range(len(tokenized_answers)) :
    # Removing "samaapt" from the answer
    tokenized_answers[i] = tokenized_answers[i][:-1] 
    
# Find max len of answers    
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )

# Pad to the max length
padded_answers = tensorflow.keras.utils.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
# Print shapes
print( decoder_input_data.shape , maxlen_answers )

(13662, 42) 42


In [17]:

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers_with_tags )
for i in range(len(tokenized_answers)) :
    # removing "prarambh" from each answer
    tokenized_answers[i] = tokenized_answers[i][1:] 

# Pad to the max length
padded_answers = tensorflow.keras.utils.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )

decoder_output_data = np.array( padded_answers )
# Print shapes
print(decoder_output_data.shape , maxlen_answers )


(13662, 42) 42


In [14]:
# https://machinelearningmastery.com/faq/single-faq/how-is-data-processed-by-an-lstm/
# https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/ 

# n_input is maxlen_questions and n_output is maxlen_answers

# Here is a question from MachineLearningMastery
# 3 models were instantiated (train, infenc, infdec). 
# The define_models() function, you can see that model weights are shared between the models – but each model provides a different use case.

def define_models(n_input, n_output, n_units):
    # 1. define training encoder
    encoder_inputs = tf.keras.layers.Input(shape=( n_input , )) 
    encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 64 , mask_zero=True ) (encoder_inputs) 
    encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( n_units , return_state=True )( encoder_embedding )
    encoder_states = [ state_h , state_c ]

    # 2. define training decoder

    decoder_inputs = tf.keras.layers.Input(shape=( n_output ,  ))
    decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 64 , mask_zero=True) (decoder_inputs) #Passing the input_length is optional here, but make use that the input_length is made constant by padding
    decoder_lstm = tf.keras.layers.LSTM( n_units , return_state=True , return_sequences=True )
    decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
    decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
    decoder_outputs = decoder_dense(decoder_outputs)

    model = tf.keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs )


    
    return model

In [71]:
# https://machinelearningmastery.com/faq/single-faq/how-is-data-processed-by-an-lstm/
# https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/ 

# n_input is maxlen_questions and n_output is maxlen_answers

# Here is a question from MachineLearningMastery
# 3 models were instantiated (train, infenc, infdec). 
# The define_models() function, you can see that model weights are shared between the models – but each model provides a different use case.

def define_models(n_input, n_output, n_units):
    # 1. define training encoder
    encoder_inputs = tf.keras.layers.Input(shape=( n_input , )) 
    encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 64 , mask_zero=True ) (encoder_inputs) 
    encoder_outputs , state_h , state_c = tf.keras.layers.LSTM(n_units , return_state=True )( encoder_embedding )
    encoder_states = [state_h,state_c]

    # 2. define training decoder

    decoder_inputs = tf.keras.layers.Input(shape=( n_output ,  ))
    decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 64 , mask_zero=True) (decoder_inputs) #Passing the input_length is optional here, but make use that the input_length is made constant by padding
    decoder_lstm = tf.keras.layers.LSTM( n_units , return_state=True , return_sequences=True )
    decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
    decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
    decoder_outputs = decoder_dense ( decoder_outputs )

    model = tf.keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs )

    # 3. Define Inference Encoder
        # SHAPES : encoder_inputs- (1, 154) || encoder_states - (1, 100)
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)

    # 4. Define Inference Decoder
    decoder_state_input_h = tf.keras.layers.Input(shape=( n_units ,)) 
    decoder_state_input_c = tf.keras.layers.Input(shape=( n_units ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_inputs_inference = tf.keras.layers.Input(shape=(1,))
    decoder_embedding_layer = tf.keras.layers.Embedding( VOCAB_SIZE, 64 , mask_zero=True )
    decoder_inputs_embedding_inference = decoder_embedding_layer(decoder_inputs_inference)
    
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs_embedding_inference , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    
    # There will be 3 Inputs of SHAPES: decoder_inputs_inference-(1,1) || decoder_states_inputs- (1,100) , (1,100)
    # There will be 3 outputs of SHAPES: decoder_outputs- (1,1,VOCAB_SIZE) || decoder_states - (1,100) , (1,100)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs_inference] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return model, encoder_model, decoder_model

In [72]:

# 3 models were instantiated (train, infenc, infdec)

train, infenc, infdec = define_models(maxlen_questions, maxlen_answers, 100)
train.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='sparse_categorical_crossentropy')
train.summary()

Model: "model_19"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_43 (InputLayer)          [(None, 55)]         0           []                               
                                                                                                  
 input_44 (InputLayer)          [(None, 42)]         0           []                               
                                                                                                  
 embedding_20 (Embedding)       (None, 55, 64)       1926016     ['input_43[0][0]']               
                                                                                                  
 embedding_21 (Embedding)       (None, 42, 64)       1926016     ['input_44[0][0]']               
                                                                                           

In [74]:
# https://machinelearningmastery.com/tensorflow-tutorial-deep-learning-with-tf-keras/
# Early stopping for avoiding overtraining of Neural Network

from tensorflow.keras.callbacks import EarlyStopping
import os
if not os.path.exists('../model/model_train.h5'):
    es = EarlyStopping(monitor='val_loss', patience=5)
    history = train.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=64, epochs=1 , validation_split=0.20, callbacks=[es])
    train.save('../model/model_train.h5') 
    infenc.save('../model/model_infenc.h5') 
    infdec.save('../model/model_infdec.h5') 

171/171 [==============================] - 138s 747ms/step - loss: 7.2009 - val_loss: 6.2068


In [15]:

# 3 models were instantiated (train, infenc, infdec)

train = define_models(maxlen_questions, maxlen_answers, 100)
train.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='sparse_categorical_crossentropy')
train.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 55)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 42)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 55, 64)       1926016     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 42, 64)       1926016     ['input_2[0][0]']                
                                                                                              

In [1]:
# Create TFLite model for infenc model
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(train)
tf_lite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = tf_lite_converter.convert()

# Write the model to the specified path
TF_LITE_MODEL_FILE_NAME_1 = "../model/tf_lite_infenc.tflite"
tflite_model_name_1 = TF_LITE_MODEL_FILE_NAME_1
open(tflite_model_name_1, "wb").write(tflite_model)

In [22]:
def str_to_tokens( sentence : str ):
    tokens = tokenizer.texts_to_sequences([sentence])
    return tensorflow.keras.utils.pad_sequences( tokens , maxlen=maxlen_questions , padding='post')

In [ ]:
def one_hot_decode(encoded_seq):
    li = [argmax(vector) for vector in encoded_seq]
    # li will be a list like [12,2230,2345......(n_steps) times]
    decoded_translation = ''
    for value in li:
        for word , index in tokenizer.word_index.items() :
            if value == index:
                decoded_translation += ' {}'.format( word )

    return decoded_translation

In [ ]:
from numpy import array,argmax
def predict_answer(n_steps):
    interpreter_1 = tf.lite.Interpreter(model_path = TF_LITE_MODEL_FILE_NAME_1)
    input_details_1 = interpreter_1.get_input_details()
    output_details_1 = interpreter_1.get_output_details()

    interpreter_2 = tf.lite.Interpreter(model_path = TF_LITE_MODEL_FILE_NAME_2)
    input_details_2 = interpreter_2.get_input_details()
    output_details_2 = interpreter_2.get_output_details()

    # Encoder Input
    li = str_to_tokens( input( 'Enter question : ' ) )

    # Changing the input to float32 bcoz the encoder input expects the type to be float32
    li2= np.array(li, dtype=np.float32)

    # Allocate tensor, set tensor, invoke, get tensor
    interpreter_1.allocate_tensors()
    interpreter_1.set_tensor(input_details_1[0]['index'], li2)

    interpreter_1.invoke()

    # Encoder Predictions
    # SHAPES: tflite_model_predictions_1_0 - (1,100) || tflite_model_predictions_1_1 - (1,100)
    tflite_model_predictions_1_0 = interpreter_1.get_tensor(output_details_1[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions_1_0.shape)

    tflite_model_predictions_1_1 = interpreter_1.get_tensor(output_details_1[1]['index'])
    #print("Prediction results shape:", tflite_model_predictions_1_1.shape)

    output = list()
    #n_steps = 10

    # First input to the decoder
    target_seq = np.zeros( ( 1 , 1 ), dtype=np.float32 ) 
    target_seq[0, 0] = tokenizer.word_index['prarambh']

    for _ in range(n_steps):
        # Allocate(only once), [set, invoke , get]
        # First Input for the Decoder of shape (1, 100)
        interpreter_2.allocate_tensors()
        interpreter_2.set_tensor(input_details_2[0]['index'], tflite_model_predictions_1_0)

        # Second Input for Decoder of shape (1, 100)
        interpreter_2.set_tensor(input_details_2[1]['index'], tflite_model_predictions_1_1)

        # Third input Decoder for Decoder of shape (1, 1)
        interpreter_2.set_tensor(input_details_2[2]['index'], target_seq)

        interpreter_2.invoke()

        # Decoder Predictions
        # SHAPES:  tflite_model_predictions_2_0- (1,1,VOCAB_SIZE) || tflite_model_predictions_2_1- (1,100) || tflite_model_predictions_2_2- (1,100) 
        tflite_model_predictions_2_0 = interpreter_2.get_tensor(output_details_2[0]['index'])
        tflite_model_predictions_2_1 = interpreter_2.get_tensor(output_details_2[1]['index'])
        tflite_model_predictions_2_2 = interpreter_2.get_tensor(output_details_2[2]['index'])

        output.append(tflite_model_predictions_2_0[0, 0, :])
        # Getting data ready for next iteration
        index = np.argmax( tflite_model_predictions_2_0[0, 0, :] )
        if index != tokenizer.word_index['samaapt']: # At a given iteration if the prediction is 'end', then the loop should break
            target_seq = np.zeros( ( 1 , 1 ), dtype=np.float32 )  
            target_seq[ 0 , 0 ] = index
        else:
            break
        
        tflite_model_predictions_1_0 = tflite_model_predictions_2_1
        tflite_model_predictions_1_1 = tflite_model_predictions_2_2

    answer_out = one_hot_decode(array(output))
    print("Answer is : ", answer_out)